In [198]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=17)

def parses(input):
    return [[int(i) for i in row] for row in input.strip().split('\n')]

# import re
# def parses(input):
#     return [int(re.findall('\d', line)) for line in nput.strip().split('\n')]

data = parses(puzzle.input_data)

In [199]:
sample = parses("""2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533""")

In [3]:
from heapq import heappush, heappop
DELTAS = {'>': 1j, '<': -1j, '^': -1, 'v': 1, '.': 0}
DIRECTIONS = {v:k for k, v in DELTAS.items()}

In [223]:
# from heapq import heappush, heappop

# def best_path(data, least, most):
    
#     visited = set()
#     N, M = len(data), len(data[0])
#     heap =[(None, 0, (0,0),(0,0),0)]
    
#     while heap:
#         _, cost, (i,j), (di,dj), momentum = heappop(heap)
        
#         if momentum == 0:
#             valid_deltas = [(1,0), (0,1)]
#         elif momentum < least:
#             valid_deltas = [(di,dj)]
#         elif momentum < most:
#             valid_deltas = [(di,dj), (dj,di), (-dj, -di)]
#         else:
#             valid_deltas = [(dj,di), (-dj, -di)]
        
#         for di2, dj2 in valid_deltas:
#             i2, j2 = i+di2, j+dj2
#             if not (0<=i2<N and 0<=j2<M):
#                 continue
                
#             new_cost = cost + data[i2][j2]
#             new_bound = new_cost + abs(N-1-i2) + abs(M-1-j2)
#             new_momentum = momentum + 1 if (di,dj) == (di2,dj2) else 1
#             new_state = (new_bound, new_cost, (i2,j2), (di2,dj2), new_momentum)
            
#             if (i2,j2, di2,dj2, new_momentum) not in visited:
#                 if (i2,j2) == (N-1, M-1):
#                     return new_cost
#                 visited.add((i2,j2, di2,dj2, new_momentum))
#                 heappush(heap, new_state)

In [261]:
from heapq import heappush, heappop

def best_path(data, least, most):
    
    visited = set()
    N, M = len(data), len(data[0])
    heap =[(0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 1j, 0)]
    
    map_ = {i+1j*j: data[i][j] for i in range(N) for j in range(M)}

    k = 0
    while heap:
        _, cost, _, pos, delta, momentum = heappop(heap)
        
        if pos == (N-1)+1j*(M-1):
            return cost
        if (pos, delta,momentum) in visited:
            continue
        visited.add((pos,delta,momentum))

        if momentum < least:
            valid_deltas = [delta]
        elif momentum < most:
            valid_deltas = [delta, 1j*delta, -1j*delta]
        else:
            valid_deltas = [1j*delta, -1j*delta]
        
        for new_delta in valid_deltas:
            new_pos = pos + new_delta
            if new_pos not in map_: 
                continue
                     
            new_cost = cost+map_[new_pos]
            new_bound = new_cost + abs(N-1-new_pos.real) + abs(M-1-new_pos.imag)
            new_direction = DIRECTIONS[delta]
            new_momentum = momentum + 1 if delta == new_delta else 1
            new_state = (new_bound, new_cost, (k := k+1), new_pos, new_delta, new_momentum)
            
            if (new_pos, new_delta) not in visited:
                heappush(heap, new_state)
                
def solve_a(data):
    return best_path(data, 1, 3)

def solve_b(data):
    return best_path(data, 4, 10)

In [262]:
best_path(sample, 1, 3)

102

In [263]:
%%time
best_path(data, 1, 3)

CPU times: user 624 ms, sys: 4.05 ms, total: 628 ms
Wall time: 633 ms


814

In [264]:
best_path(sample, 4, 10)

94

In [265]:
%%time
best_path(data, 4, 10)

CPU times: user 1.92 s, sys: 3.78 ms, total: 1.92 s
Wall time: 1.94 s


974

In [216]:
from heapq import heappush, heappop

def best_path(data, delta_fn):

    DELTAS = {'>': 1j, '<': -1j, '^': -1, 'v': 1, '.': 0}
    DIRECTIONS = {v:k for k, v in DELTAS.items()}
    visited = set()
    N, M = len(data), len(data[0])
    heap =[(0, 0,(0,0),'.',0)]
    
    while heap:
        _, cost, (i,j), direction, momentum = heappop(heap)
        delta = DELTAS[direction]
        valid_deltas = delta_fn(delta, momentum)
        
        for delta in valid_deltas:
            new_pos = (i + 1j *j) + delta
            i2,j2 = int(new_pos.real), int(new_pos.imag)
            if not (0<=i2<N and 0<=j2<M):
                continue
                
            new_cost = cost+data[i2][j2]
            new_bound = new_cost + abs(N-1-i2) + abs(M-1-j2)
            new_direction = DIRECTIONS[delta]
            new_momentum = momentum + 1 if direction == new_direction else 1
            new_state = (new_bound, new_cost, (i2,j2), new_direction, new_momentum)
            
            if new_state[2:] not in visited:
                if (i2,j2) == (N-1, M-1):
                    return new_state[1]
                visited.add(new_state[2:])
                heappush(heap, new_state)
                
def solve_a(data):
    def valid_deltas(delta, momentum):
        if momentum == 0: return [-1,1,1j,-1j]
        if momentum < 3: return [delta, delta*1j, delta*-1j]
        else: return [delta*1j, delta*-1j]
    return best_path(data, valid_deltas)

def solve_b(data):
    def valid_deltas(delta, momentum):
        if momentum == 0: return [-1,1,1j,-1j]
        if momentum < 4: return [delta]
        if momentum < 10: return [delta, delta*1j, delta*-1j]
        else: return [delta*1j, delta*-1j]
    return best_path(data, valid_deltas)

In [221]:
%%time
solve_a(data)

CPU times: user 427 ms, sys: 2.81 ms, total: 430 ms
Wall time: 433 ms


814

In [222]:
%%time
solve_b(data)

CPU times: user 1.42 s, sys: 3.91 ms, total: 1.43 s
Wall time: 1.44 s


974

In [12]:
import itertools
def solve_a(data):
    visited = set()
    heap =[(0, 0,(0,0),'.',0)]

    N, M = len(data), len(data[0])
    for k in itertools.count():
        _, cost, (i,j), direction, momentum = heappop(heap)
        pos = i + 1j *j

        delta = DELTAS[direction]
        if momentum == 3:
            valid_deltas = [delta*1j, delta*-1j]
        else:
            valid_deltas = [d for d in (1,-1,1j,-1j) if d != -delta]
            
        for delta in valid_deltas:
            new_pos = pos + delta

            i2,j2 = int(new_pos.real), int(new_pos.imag)
            if not (0<=i2<N and 0<=j2<M):
                continue
            
            
            new_cost = cost+data[i2][j2]
            new_bound = new_cost + abs(N-1-i2) + abs(M-1-j2)
            new_direction = DIRECTIONS[delta]
            new_momentum = momentum + 1 if direction == new_direction else 1
            
                
            new_state = (new_bound, new_cost, (i2,j2), new_direction, new_momentum)

            if new_state[2:] not in visited:

                if (i2,j2) == (N-1, M-1):
                    return new_state[1]

                visited.add(new_state[2:])
                heappush(heap, new_state)
#         print(k, heap)
#         print()
#         if k == 2:
#             break

In [13]:
solve_a(sample)

102

In [14]:
solve_a(data)

814

In [15]:
import itertools
def solve_b(data):
    visited = set()
    heap =[(0, 0,(0,0),'.',0)]

    N, M = len(data), len(data[0])
    for k in itertools.count():
        _, cost, (i,j), direction, momentum = heappop(heap)
        pos = i + 1j *j

        delta = DELTAS[direction]
        if momentum == 0:
            valid_deltas = [1,-1,1j,-1j]
        elif momentum < 4:
            valid_deltas = [delta]
        elif momentum < 10:
            valid_deltas = [delta, delta*1j, delta*-1j]
        else:
            valid_deltas = [delta*1j, delta*-1j]
            
        for delta in valid_deltas:
            new_pos = pos + delta

            i2,j2 = int(new_pos.real), int(new_pos.imag)
            if not (0<=i2<N and 0<=j2<M):
                continue
            
            
            new_cost = cost+data[i2][j2]
            new_bound = new_cost + abs(N-1-i2) + abs(M-1-j2)
            new_direction = DIRECTIONS[delta]
            new_momentum = momentum + 1 if direction == new_direction else 1
            
                
            new_state = (new_bound, new_cost, (i2,j2), new_direction, new_momentum)

            if new_state[2:] not in visited:

                if (i2,j2) == (N-1, M-1):
                    return new_state[1]

                visited.add(new_state[2:])
                heappush(heap, new_state)
#         print(k, heap)
#         print()
#         if k == 2:
#             break

In [16]:
solve_b(sample)

94

In [17]:
solve_b(data)

974

In [15]:
def solve_a(data):
    pass

In [2]:
def solve_b(data):
    pass